In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the tickers and download the data
tickers = ['2330.TW', '2317.TW', '2454.TW', '2881.TW', '2382.TW',
           '2308.TW', '2882.TW', '2412.TW', '2891.TW', '3711.TW']

data = yf.download(tickers=tickers, interval='1mo', start='2019-01-01', end='2023-12-31')


[*********************100%%**********************]  10 of 10 completed


In [2]:
# Filter data for 2021
data_2021 = data['Adj Close'].loc['2021-01-01':'2021-12-31']

# Find the minimum price
lowest_price = data_2021.min().min()
print(f"Lowest price in 2021: {lowest_price:.5f}")


Lowest price in 2021: 15.76368


In [3]:
# Get the price data for 2021-01-01
# Since it's monthly data, we want January 2021
mask_date = (data.index.year == 2021) & (data.index.month == 1)
prices_jan2021 = data.loc[mask_date, 'Adj Close']

# For price-weighted index with divisor = 1
# Simply sum up all prices
pw_index = prices_jan2021.iloc[0].sum()

print(f"Price-weighted index value on 2021-01-01: {pw_index:.2f}")

# Optional: Display individual stock prices for verification
print("\nIndividual stock prices:")
print(prices_jan2021.iloc[0])

Price-weighted index value on 2021-01-01: 1855.64

Individual stock prices:
Ticker
2308.TW    258.548859
2317.TW     95.108681
2330.TW    547.284668
2382.TW     63.300213
2412.TW     92.765083
2454.TW    645.098022
2881.TW     30.741611
2882.TW     33.127731
2891.TW     15.763680
3711.TW     73.905174
Name: 2021-01-01 00:00:00, dtype: float64


In [4]:
# Calculate percentage returns for 2412.TW
returns_2412 = data['Adj Close']['2412.TW'].pct_change().dropna()

# Compute average percentage return
avg_return_2412 = returns_2412.mean()
print(f"Average percentage return for 2412.TW: {avg_return_2412:.6f}")


Average percentage return for 2412.TW: 0.005587


In [5]:
# Log return for 2881.TW on 2019-12-01
price_2019_12_01 = data['Adj Close']['2881.TW']['2019-12-01']
price_prev = data['Adj Close']['2881.TW']['2019-11-01']

log_return = np.log(price_2019_12_01 / price_prev)
print(f"Log return of 2881.TW on 2019-12-01: {log_return:.6f}")


Log return of 2881.TW on 2019-12-01: 0.035091


In [6]:
# Percentage returns for 2412.TW and 2382.TW
returns_2412 = data['Adj Close']['2412.TW'].pct_change().dropna()
returns_2382 = data['Adj Close']['2382.TW'].pct_change().dropna()

# Compute covariance
covariance = np.cov(returns_2412, returns_2382)[0, 1]
print(f"Covariance between 2412.TW and 2382.TW: {covariance:.6f}")


Covariance between 2412.TW and 2382.TW: -0.000118


### GMVP & MSRP 

In [7]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Compute monthly returns
returns = data['Adj Close'].pct_change().dropna()

# Calculate covariance matrix
cov_matrix = returns.cov()

# Number of assets
num_assets = len(returns.columns)

# Objective function: Minimize portfolio variance
def portfolio_variance(weights, cov_matrix):
    return weights.T @ cov_matrix @ weights

# Constraint: Sum of weights = 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

# Bounds: Allow short selling (weights can be negative)
bounds = [(None, None) for _ in range(num_assets)]

# Initial guess: Equal weights
init_guess = np.ones(num_assets) / num_assets

# Minimize variance with short selling allowed
result_gmvp = minimize(
    portfolio_variance,
    init_guess,
    args=(cov_matrix,),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints,
    options={'ftol': 1e-20, 'disp': True}
)

# Extract GMVP weights
gmvp_weights = result_gmvp.x

# Get weight for 2882.TW
weight_2882 = gmvp_weights[list(returns.columns).index('2882.TW')]

print("GMVP Weights (Short Selling Allowed):", gmvp_weights)
print("Weight on 2882.TW:", weight_2882)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0005088734305905677
            Iterations: 73
            Function evaluations: 821
            Gradient evaluations: 73
GMVP Weights (Short Selling Allowed): [-0.07111991  0.11040897  0.2105858   0.06856821  0.89462672 -0.07552467
  0.16916632 -0.20209939 -0.06321358 -0.04139848]
Weight on 2882.TW: -0.20209938551363618


In [8]:
gmvp_variance = result_gmvp.fun

In [9]:
np.sqrt(result_gmvp.fun)

np.float64(0.02255822312573771)

In [10]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Compute monthly returns
returns = data['Adj Close'].pct_change().dropna()

# Calculate covariance matrix and mean returns
cov_matrix = returns.cov()
mean_returns = returns.mean()

# Number of assets
num_assets = len(mean_returns)

# Objective function: Negative Sharpe ratio
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0):
    portfolio_return = weights.T @ mean_returns
    portfolio_std_dev = np.sqrt(weights.T @ cov_matrix @ weights)
    return -(portfolio_return - risk_free_rate) / portfolio_std_dev

# Constraint: Sum of weights = 1
constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

# Bounds: Allow short selling (weights can be negative)
bounds = [(None, None) for _ in range(num_assets)]

# Initial guess: Equal weights
init_guess = np.ones(num_assets) / num_assets

# Minimize negative Sharpe ratio with short selling allowed
result_msrp = minimize(
    neg_sharpe_ratio,
    init_guess,
    args=(mean_returns, cov_matrix),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints,
    options={'ftol': 1e-20, 'disp': True}
)

# Extract MSRP weights
msrp_weights = result_msrp.x

# Get weight for 2882.TW
weight_2882 = msrp_weights[list(returns.columns).index('2882.TW')]

print("MSRP Weights (Short Selling Allowed):", msrp_weights)
print("Weight on 2882.TW:", weight_2882)


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.6897068834022968
            Iterations: 27
            Function evaluations: 341
            Gradient evaluations: 27
MSRP Weights (Short Selling Allowed): [-0.22222979  0.25558117  0.2959635   0.22787319  0.75761081  0.06754257
  1.19116236 -1.3923752  -0.06146903 -0.11965958]
Weight on 2882.TW: -1.3923751983604604


In [11]:
msrp_weights

array([-0.22222979,  0.25558117,  0.2959635 ,  0.22787319,  0.75761081,
        0.06754257,  1.19116236, -1.3923752 , -0.06146903, -0.11965958])

In [12]:
mean_returns

Ticker
2308.TW    0.017962
2317.TW    0.012968
2330.TW    0.022768
2382.TW    0.035165
2412.TW    0.005587
2454.TW    0.036774
2881.TW    0.015583
2882.TW    0.006785
2891.TW    0.011309
3711.TW    0.023359
dtype: float64

In [13]:
# Calculate the expected return of the MSRP
msrp_expected_return = np.dot(msrp_weights, mean_returns)

print(f"In-sample expected return of the MSRP: {msrp_expected_return:.6f}")


In-sample expected return of the MSRP: 0.026415


In [21]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# First, get the data as specified in the exam
tickers = ['2330.TW', '2317.TW', '2454.TW', '2881.TW', '2382.TW',
           '2308.TW', '2882.TW', '2412.TW', '2891.TW', '3711.TW']

data = yf.download(tickers=tickers, interval='1mo', start='2019-01-01', end='2023-12-31')

# Calculate returns
returns = data['Adj Close'].pct_change().dropna()

# Calculate mean returns and covariance matrix
mean_returns = returns.mean()
cov_matrix = returns.cov()

def portfolio_stats(weights, mean_returns, cov_matrix):
    """Calculate portfolio statistics"""
    portfolio_return = np.sum(mean_returns * weights)
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_return, portfolio_std

def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0):
    """Calculate negative Sharpe ratio for minimization"""
    p_ret, p_std = portfolio_stats(weights, mean_returns, cov_matrix)
    return -(p_ret - risk_free_rate) / p_std

def gmvp_objective(weights, cov_matrix):
    """Objective function for GMVP - minimizes variance"""
    return np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

# Constraints
def constraint_sum_to_one(weights):
    return np.sum(weights) - 1

# GMVP Optimization (unconstrained)
n_assets = len(tickers)
initial_weights = np.array([1/n_assets] * n_assets)
bounds = None  # Allow short selling
constraints = {'type': 'eq', 'fun': constraint_sum_to_one}
options = {'ftol': 1e-20, 'disp': True}

# Question 12: GMVP margin calculation
gmvp_result = minimize(gmvp_objective, initial_weights, args=(cov_matrix,),
                      method='SLSQP', constraints=constraints, bounds=bounds, options=options)
gmvp_weights = gmvp_result.x
gmvp_margin = np.sum(np.abs(gmvp_weights))
print(f"GMVP Margin: {gmvp_margin}")

[*********************100%%**********************]  10 of 10 completed

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.022558223125737718
            Iterations: 38
            Function evaluations: 424
            Gradient evaluations: 38
GMVP Margin: 1.90671200926251


In [22]:
def utility_objective(weights, mean_returns, cov_matrix, risk_aversion):
    """Objective function for utility maximization"""
    p_ret, p_std = portfolio_stats(weights, mean_returns, cov_matrix)
    return -(p_ret - (risk_aversion/2) * (p_std**2))

A = 15  # Risk aversion parameter
ocp_result = minimize(utility_objective, initial_weights, 
                     args=(mean_returns, cov_matrix, A),
                     method='SLSQP', constraints=constraints, bounds=bounds, options=options)
ocp_weights = ocp_result.x
ocp_return, ocp_std = portfolio_stats(ocp_weights, mean_returns, cov_matrix)
ocp_sharpe = ocp_return / ocp_std  # Risk-free rate = 0
print(f"OCP Expected Return (A=15): {ocp_return}")
print(f"OCP Sharpe Ratio: {ocp_sharpe}")

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.01570298457203299
            Iterations: 78
            Function evaluations: 1043
            Gradient evaluations: 78
OCP Expected Return (A=15): 0.029874989995624452
OCP Sharpe Ratio: 0.6872626546181392


In [23]:
# Question 15: Certainty Equivalent with A=2.5
A_ce = 2.5
ce_result = minimize(utility_objective, initial_weights,
                    args=(mean_returns, cov_matrix, A_ce),
                    method='SLSQP', constraints=constraints, bounds=bounds, options=options)
ce_weights = ce_result.x
ce_return, ce_std = portfolio_stats(ce_weights, mean_returns, cov_matrix)
certainty_equivalent = ce_return - (A_ce/2) * (ce_std**2)
print(f"Certainty Equivalent (A=2.5): {certainty_equivalent}")

Iteration limit reached    (Exit mode 9)
            Current function value: -0.07066071989278663
            Iterations: 100
            Function evaluations: 1332
            Gradient evaluations: 100
Certainty Equivalent (A=2.5): 0.07066071989278663


In [24]:
# Question 16: Constrained GMVP (no short selling)
bounds_no_short = tuple((0, None) for _ in range(n_assets))
gmvp_constrained = minimize(gmvp_objective, initial_weights, args=(cov_matrix,),
                          method='SLSQP', constraints=constraints, bounds=bounds_no_short, options=options)
gmvp_constrained_weights = gmvp_constrained.x
print(f"Constrained GMVP weight for 2881.TW: {gmvp_constrained_weights[3]}")

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.02510474646464375
            Iterations: 25
            Function evaluations: 237
            Gradient evaluations: 21
Constrained GMVP weight for 2881.TW: 0.03477998600770112
